In [1]:
%autosave 30
%load_ext autoreload
%autoreload 2

Autosaving every 30 seconds


In [2]:
cd ..

/home/huawei123/kwx1991442/code-classification


In [3]:
import logging
from typing import *
import pickle
import os

import numpy as np
from torch import Generator
from torch.utils.data import random_split, DataLoader
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support, roc_auc_score

from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB, BernoulliNB
from liblinear.liblinearutil import (
    problem, parameter, train, predict, evaluations, save_model
)
from src.utils import CustomSummaryWriter
from src.baseline.dataloader import UITestsDataset
from src.word2vec.dataloader import W2VClassificationCollator, TrainW2VModel
from src.params import (PATH_TEST_UI, PATH_PARSED_CLASSIFUI,
    PATH_SAVE_METRICS, PATH_SAVE_MODEL, PATH_SAVE_PREDICTIONS)


logger = logging.getLogger(__name__)

/home/huawei123/kwx1991442/venv/lib/python3.8/site-packages/liblinear/liblinear.py:135: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def csr_to_problem_jit(l, x_val, x_ind, x_rowptr, prob_val, prob_ind, prob_rowptr):
2023-08-25 15:38:19.067501: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-25 15:38:20.583352: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [4]:
from src.main import parse_args

args = parse_args(True, ["--method", "word2vec", "--exp-name", "DEBUGDOC2VEC", '--tokens-source', 'origin'])

In [5]:
if args.mode != 'eval':
    raise NotImplementedError()

# load data
mode = 'train'
tokens_source = args.tokens_source

if tokens_source == 'origin':
    tests_ui_folder=PATH_TEST_UI
elif tokens_source == 'classifui':
    tests_ui_folder=PATH_PARSED_CLASSIFUI

dataset = UITestsDataset(tests_ui_folder, mode, debug=True)
logger.info(f"UITestsDataset is initialized in mode {mode} from {tests_ui_folder} "
            f" with {tokens_source=}. {len(dataset)=}")

In [6]:
from src.word2vec.dataloader import TokenizerSplitter

splitter = TokenizerSplitter()
sentences_train = [splitter(text) for text, _ in dataset]

In [7]:
from gensim.models import doc2vec
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

tag_documents = [TaggedDocument(sentences_train[i], [i]) for i in range(len(sentences_train))]

In [8]:
tag_documents[0]


TaggedDocument(words=['vari', 'adic', 'Ġffi', 'Ġ6', 'Ċ', 'Ċ', '#!', '[', 'cr', 'ate', '_', 'type', '="', 'lib', '"]', 'Ċ', '#!', '[', 'feature', '(', 'c', '_', 'vari', 'adic', ')]', 'Ċ', 'Ċ', 'pub', 'Ġunsafe', 'Ġextern', 'Ġ"', 'C', '"', 'Ġfn', 'Ġuse', '_', 'var', 'arg', '_', 'lifetime', '(', 'ĊĠĠĠ', 'Ġx', ':', 'Ġu', 'size', ',', 'ĊĠĠĠ', 'Ġy', ':', 'Ġ...', 'Ċ', ')', 'Ġ->', 'Ġ&', 'u', 'size', 'Ġ{', 'Ġ//', '~', 'ĠERROR', 'Ġmissing', 'Ġlifetime', 'Ġspecifier', 'ĊĠĠĠ', 'Ġ&', '0', 'Ċ', '}', 'Ċ', 'Ċ', 'pub', 'Ġunsafe', 'Ġextern', 'Ġ"', 'C', '"', 'Ġfn', 'Ġuse', '_', 'normal', '_', 'arg', '_', 'lifetime', '(', 'x', ':', 'Ġ&', 'u', 'size', ',', 'Ġy', ':', 'Ġ...)', 'Ġ->', 'Ġ&', 'u', 'size', 'Ġ{', 'Ġ//', 'ĠOK', 'ĊĠĠĠ', 'Ġx', 'Ċ', '}', 'ĊĊ', 'Ċ', 'error', '[', 'E', '01', '06', ']:', 'Ġmissing', 'Ġlifetime', 'Ġspecifier', 'ĊĠ', 'Ġ-->', 'Ġ$', 'DIR', '/', 'vari', 'adic', '-', 'ffi', '-', '6', '.', 'rs', ':', '7', ':', '6', 'ĊĠĠ', 'Ġ|', 'Ċ', 'LL', 'Ġ|', 'Ġ)', 'Ġ->', 'Ġ&', 'u', 'size', 'Ġ{', 'ĊĠĠ', 'Ġ|'

In [9]:
model = doc2vec.Doc2Vec(vector_size = 300, window = 1, min_count = 3, workers = 6, epochs=20)
model.build_vocab(tag_documents)
model.train(tag_documents, total_examples=model.corpus_count, epochs=20)

In [19]:
from sklearn.preprocessing import LabelEncoder
from src.word2vec.dataloader import LiblinearSplitter, TokenizerSplitter

class D2VClassificationCollator(object):
    d2v_model : Doc2Vec
    labelencoder : LabelEncoder
    splitter : Union[LiblinearSplitter, TokenizerSplitter]
    
    def __init__(self, 
        d2v_model : Doc2Vec,
        splitter : Union[LiblinearSplitter, TokenizerSplitter], 
        classes : List[str]
        ) -> None:
        self.d2v_model = d2v_model
        self.splitter = splitter
        self.labelencoder = LabelEncoder().fit(classes)

    def encode_sentence(self, sentence):
        if len(sentence) == 0:
            return np.zeros(self.w2v_model.vector_size)        
        else:
            return self.d2v_model.infer_vector(sentence)

    def encode_labels(self, labels: List[str]) -> np.ndarray:
        return self.labelencoder.transform(labels)

    def __call__(self, rawtexts_labels : List[Tuple[str, str]]) -> Tuple[np.ndarray, np.ndarray]:
        rawtexts, labels = zip(*rawtexts_labels)
        vectors = [self.encode_sentence(self.splitter(text)) for text in rawtexts]
        encoded_labels = self.encode_labels(labels)
        return vectors, encoded_labels

In [15]:
# model.infer_vector(sentences_train[0])
seed = args.seed
traintestsplit = args.traintestsplit
lengths = [traintestsplit, 1-traintestsplit]

splitgenerator = Generator().manual_seed(seed)
train_set, val_set = random_split(dataset, 
    lengths=lengths, generator=splitgenerator)
logger.info(f"Split generator initialized with {seed=}, {lengths=}.")

In [20]:
collate_fn = D2VClassificationCollator(model, splitter, dataset.classes)
X_train, Y_train = collate_fn(train_set)
X_val,   Y_val   = collate_fn(val_set)
logger.info(f"{len(dataset.classes)=}, {len(X_train)=}, {len(X_val)=}")

In [24]:
args.save_metrics = True

In [25]:
from src.word2vec.trainer import classifiers, Trainer

experiment_name = f"TESTD2V"

available_classifiers = list(classifiers.keys())
selected_classifier = args.classifier
logtb = True

for name in available_classifiers:
    if selected_classifier in name or selected_classifier == 'all':
        logger.info(f"Starting {name} classifier")
        training = Trainer(name, 
            logtb, 
            experiment_name, 
            labelencoder=collate_fn.labelencoder, 
            save_model=args.save_model,
            save_metrics=args.save_metrics,
            save_predictions=args.save_predictions
        )
        logger.info(f"Fitting {name} classifier")
        training.fit(X_train, Y_train)
        logger.info(f"Evaluation of {name} classifier")
        training.eval(X_val, Y_val)

/home/huawei123/kwx1991442/venv/lib/python3.8/site-packages/sklearn/model_selection/_split.py:725: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


IndexError: index 20 is out of bounds for axis 0 with size 20

In [36]:
from src.word2vec.trainer import evaluate

Y_pred = training.clf.best_estimator_.predict(X_val)
metrics = evaluate(Y_pred, Y_val, collate_fn.labelencoder)

In [37]:
m_vals, m_arrs, m_count = metrics
metrics

({'accuracy': 0.7450980392156863,
  'micro_precision': 0.7450980392156863,
  'micro_recall': 0.7450980392156863,
  'micro_fbeta_score': 0.7450980392156863,
  'macro_precision': 0.7450980392156863,
  'macro_recall': 0.7450980392156863,
  'macro_fbeta_score': 0.7450980392156863},
 {'precision': array([0.8125    , 0.5       , 1.        , 1.        , 0.33333333,
         0.71428571, 1.        , 0.5       , 0.68965517, 0.73913043,
         1.        , 1.        , 0.5       , 0.66666667, 0.9       ,
         1.        , 1.        , 1.        , 0.74264706, 1.        ,
         0.5       ]),
  'recall': array([1.        , 1.        , 1.        , 1.        , 0.25      ,
         0.55555556, 0.75      , 1.        , 0.86956522, 0.62962963,
         1.        , 0.875     , 0.75      , 1.        , 0.5       ,
         1.        , 1.        , 1.        , 0.808     , 0.6       ,
         1.        ]),
  'fbeta_score': array([0.89655172, 0.66666667, 1.        , 1.        , 0.28571429,
         0.625  

In [35]:
len(m_count['counts_y_val']), len(m_count['counts_y_pred_x'])

(20, 20)

In [ ]:
le = collate_fn.labelencoderle

for i, (count_x, count_y) in enumerate(zip(m_count['counts_y_pred_x'], m_count['counts_y_pred'])):
    classname = f"({le.inverse_transform([count_x])[0]})"
